In [1]:
import torch
import numpy as np
from torch.autograd import Variable
from scipy.stats import qmc
import time
import shutil
import os
from scipy.stats import norm
#from torch.distributions import Normal
from datetime import datetime

from Building_Net import Net
from Conditions import lossIC, lossBdry,lossNSpde, lossNSpde_rank, total_free_e_derivative

currentDateTime = datetime.now()
print("Date of Today : ", currentDateTime.month, " /", currentDateTime.day, "\nHour : ", currentDateTime.hour) 
ctime = f"{currentDateTime.month}_{currentDateTime.day}_{currentDateTime.hour}h"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

std_log = []
load = False
loadfile = '8_6_21h_Training_4' #5_12_8h_Training_41
def create_network(load , loadfile ):
    
    net = Net()
    net = net.to(device)
    global epsilon
    global min_raw_loss
    min_raw_loss = 100
    #Attempt to load the saved pt. file
    if load == True:
        prev_time = loadfile.split('h')[0]
        try:
            data = np.loadtxt(f"{prev_time}h_epsilon.txt") #
            epsilon = data.tolist()
        except:
            epsilon = []
        try:
            net.load_state_dict(torch.load(f'{loadfile}.pt', map_location=device))
        except:
            print("\nLoading file was failed\n")
        else:
            print("\nLoading file was completed\n")
    else:
        epsilon = []
    
    print('Training PDE')
    start = time.time() #initialize tracking computational time
    
    partial_time_set = [0, 0, 0, 0, 0] #initialize time recording list with number of learnning rates on whole domain training
    
    for i in range(len(partial_time_set)):
        if i == 0:
            #First loop uses progressively increasing time intervals
            print(f'\n\nTraining Pass {i+1}')
            time_slices = np.array([.1, .2]) #.1, 
            iterations = 100 #iterations for each learning rate
            learning_rate = 1e-3 #5*1e upgrade for each
        elif i == 1:
            print(f'\n\nTraining Pass {i+1}')
            #time_slices = [time_slices[-1]]
            time_slices = np.array([.3, .4])#
            iterations = 100 #iterations for each learning rate
            learning_rate = 5*1e-4 
        elif i == 2:
            print(f'\n\nTraining Pass {i+1}')
            time_slices = np.array([ .5, .6])#.8,
            iterations = 100 #iterations for each learning rate
            learning_rate = 1e-4 
        elif i == 3:
            print(f'\n\nTraining Pass {i+1}')
            time_slices = np.array([.7, .8])#
            iterations = 100 #iterations for each learning rate
            learning_rate = 5*1e-5
        elif i == 4:
            print(f'\n\nTraining Pass {i+1}')
            time_slices = np.array([ .9, 1.0])#     
            iterations = 100 #iterations for each learning rate
            learning_rate = 1e-5 #5*10**-6
        training_loop(net, time_slices, iterations, learning_rate, record_loss = 100, print_loss = 100)
        torch.save(net.state_dict(), f"{ctime}_Training_{i+1}.pt")
        partial_time_set[i] = time.time()
        #np.savetxt(f"{ctime}epsilon_{i}.txt", epsilon)

    print("Total Time:\t", partial_time_set[-1]-start, '\nPass 1 Time:\t', partial_time_set[0]-start, 
          '\nPass 2 Time:\t', partial_time_set[1]-partial_time_set[0], '\nPass 3 Time:\t', partial_time_set[2]-partial_time_set[1],
             '\nPass 4 Time:\t', partial_time_set[3]-partial_time_set[2],'\nPass 5 Time:\t', partial_time_set[4]-partial_time_set[3])
   
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def exp_time_sample(collocation_pt_number, t_l, t_u):
    if collocation_pt_number > 6000:
        c = 0
    else:
        c = 0
    t_uni = np.random.uniform(0,1, size=(collocation_pt_number, 1))
    if t_u <=.2: #0.2
        t = t_l + t_uni*(t_u-t_l)
    else:
        if c !=0:
            r = c *(t_u-.1)/t_u #3*(t_u-.1)/t_u
            t = - np.log(1-t_uni+t_uni*np.exp(-r*(t_u-t_l)))/r
        else:
            t = t_l + t_uni*(t_u-t_l)
    
    return t
def truncated_normal(mean , std, n_samples, lower, upper) -> torch.Tensor:
    samples = []
    while len(samples) < n_samples:
        batch = torch.normal(mean = mean , std=std) #torch.randn(n_samples)
        valid = batch[(batch >= lower) & (batch <= upper)]
        samples.extend(valid.tolist())
    return torch.tensor(samples[:n_samples])
    
def normal_pdf(x, mean, std):
    return 1 / (std * np.sqrt(2 * np.pi)) * torch.exp(-0.5 * ((x - mean) / std)**2)

def batch_kde_likelihood_1d(queries, data, bandwidth):
    """
    Compute KDE likelihood for N queries against N groups of 20 points each.

    Args:
        queries (Tensor): (N, 1) query values
        data (Tensor): (N, 20) corresponding reference points
        bandwidth (float): kernel bandwidth

    Returns:
        Tensor: (N,) KDE likelihoods for each query
    """
    N = queries.shape[0]

    # Expand for broadcasting
    queries_exp = queries.expand(-1, data.shape[1])  # (N, 20)

    # Squared distances
    sq_dists = ((queries_exp - data) ** 2)  # (N, 20)

    # Gaussian kernel values
    kernel_vals = torch.exp(-0.5 * sq_dists / (bandwidth ** 2))  # (N, 20)

    # KDE normalization constant for 1D Gaussian
    const = 1 / (bandwidth * torch.sqrt(torch.tensor(2 * torch.pi)))

    # Mean over the 20 reference points
    likelihoods = const * kernel_vals.mean(dim=1)  # (N,)

    return likelihoods.reshape(-1,1)
def kde_pairwise_similarity(queries, data, bandwidth=1.0):
    """
    Gaussian kernel similarity between each row of queries and data.

    Args:
        queries (Tensor): (N, D)
        data (Tensor): (N, D)
        bandwidth (float): Bandwidth (smoothing) parameter

    Returns:
        Tensor: (N, 1) similarity scores
    """
    # Compute squared L2 distances for each row pair
    sq_dists = ((queries - data) ** 2).sum(dim=1, keepdim=True)  # (N, 1)

    # Gaussian kernel
    similarity = torch.exp(-0.5 * sq_dists / (bandwidth ** 2))  # (N, 1)

    return similarity  
#########################################################################################
def e_adaptive_sampling(net,initial_size, spatial_size, x_IC, x_dom, t, initialize_sampling, final_time):
    global epoch
    size = initial_size + spatial_size #50, 4000
    
    loss_flag = 20
    accepted_IC = accepted_dom = acceptance_rate_IC = acceptance_rate_dom = mean_std_IC = mean_std_dom= 0
    
    #pre_std = np.sqrt(0.0001)*.5
    
    cumul_accepted=0
    
    x_prev = torch.cat((x_IC, x_dom),axis = 0).to(device) ## current state theta
    zero = torch.zeros_like(x_prev).to(device)
    one = torch.ones_like(x_prev).to(device)
    num_resampling = 300 #500 if initialize_sampling == True else 200 #10000
    i = 1
    #t_IC = exponential_time_sample(spatial_size, 0, final_time, 4)
    IC_lh_sampler3 = qmc.LatinHypercube(d=1)
    PDE_lh_sampler4 = qmc.LatinHypercube(d=2)
    x_l = net.x1_l
    x_u = net.x1_u
    #----------------------parameters---------------------------------------------------------
    const_std_IC = 0.07 *(0.073/0.07)**(epoch*1/99) # 0.2 -> 0.25
    const_std_dom = 0.07 *(0.073/0.07)**(epoch*1/99)
    max_r =  0.25 #0.1 #0.08
    epsilon = 2 #0.01 #*(0.025/0.01)**(epoch*1/99) # 0.01 -> 0.025 (final time .8 ~)#0.001->0.01 (final time 0 ~ .7) #*(0.001/0.05)**(epoch*1/99) #0.001 - 0.05
    #q = 0.05*(0.15/0.05)**(epoch*1/99)
    # Radius and number of points
    n_points = 20 #changed
    update_std_iter = 10 # 500 if initialize_sampling == True else 10
    #-----------------------------------------------------------------------------------------
    
    while i <= num_resampling:
        if t != None:
            t2 = torch.cat((torch.zeros_like(x_IC), t), axis =0).to(device)
            #if final_time >= .3:
            #    t2 = torch.clamp((torch.ceil(t2.float()/(final_time/4))+1)*(final_time/4), final_time)
            t3=t2.repeat(1, n_points).reshape(-1,1).to(device)
            prob = total_free_e_derivative(net, x_prev, t2, min_raw_loss) #.to(device) #.cpu().numpy()  
            
            
        elif t == None:
            t2 = None
            prob = total_free_e_derivative(net, x_prev, t2, min_raw_loss) #.to(device)
            t3 = t2

        #if i == 1:
            #observed = net.scaler(prob)  ## observed data y
            
            #prob_e = observed
            
        #else:
        prob_e = net.scaler(prob)
        '''
        if i == num_resampling:
            print(torch.quantile(prob_e, q = 0.25, dim=0).item(),
                  torch.quantile(prob_e, q = 0.5, dim=0).item(),
                  torch.quantile(prob_e, q = 0.75, dim=0).item())
        '''
        ######Set radius around the proposed x
        
        #p_e = torch.clamp(1-prob_e,min=0)-0
        radius = max_r #- torch.log(1-p_e+p_e*np.exp(-r*(max_r-min_r)))/r #1e-2 + p_e**3 #
            
        x_prev_centered_pts_set = torch.clamp(x_prev + (torch.rand(x_prev.shape[0], n_points).to(device) * 2 * radius - radius), min= x_l,max = x_u)
        x_prev_centered_pts_set = x_prev_centered_pts_set.reshape(-1,1).to(device)
            
        x_prev_pts_set_e = total_free_e_derivative(net, x_prev_centered_pts_set, t3, min_raw_loss)
        x_prev_pts_set_e = net.scaler(x_prev_pts_set_e) 
            
        x_prev_pts_set_e = x_prev_pts_set_e.reshape(x_prev.shape[0],n_points)
        if i ==1:
            observed = x_prev_pts_set_e
        
        # Compute the maximum in each set
        
        #e_max_per_set = torch.quantile(x_prev_pts_set_e, q = q, dim=1, keepdim =True).reshape(-1,1) #.values ## simulation data x
        '''
        compliment_prob_e = one-e_max_per_set**.1
        compliment_prob_e_IC = compliment_prob_e[:initial_size]
        compliment_prob_e_dom = compliment_prob_e[initial_size:]
        '''
        prop_std_IC = const_std_IC #*compliment_prob_e_IC# **.5
        prop_std_dom = const_std_dom #*compliment_prob_e_dom #**.5
        
        prop_std = const_std_IC #torch.cat([prop_std_IC, prop_std_dom], axis = 0)
        
        
        #normal random variable generate
        #prop_std_IC = torch.max(prop_std_IC,torch.zeros_like(prop_std_IC))
        prop_x_IC = torch.normal(mean = x_prev[:initial_size] , std=prop_std_IC).to(device) ## propose thetap
        #prop_std_dom = torch.max(prop_std_dom,torch.zeros_like(prop_std_dom))
        prop_x_dom = torch.normal(mean = x_prev[initial_size:] , std=prop_std_dom).to(device) ## propose thetap
        prop_std = const_std_IC #torch.cat([prop_std_IC, prop_std_dom], axis = 0)
        prop_x = torch.cat([prop_x_IC, prop_x_dom], axis = 0)
        
        prop_x = Variable(prop_x,requires_grad=True).to(device) 
        
        
        prob_e_prop = total_free_e_derivative(net, prop_x, t2, min_raw_loss) #.cpu().numpy(),
        prob_e_prop = net.scaler(prob_e_prop)
        
        ######Set radius around the proposed x
        #p_e_rev = torch.clamp(1-prob_e_prop,min=0)-0 #.2
        radius_rev = max_r #- torch.log(1-p_e_rev+p_e_rev*np.exp(-r*(max_r-min_r)))/r #torch.clamp((1-prob_e_prop)**3, min=0.01,max=1) -0.01 #0.99 changed*
        
        prop_x_centered_pts_set = torch.clamp(prop_x + (torch.rand(prop_x.shape[0], n_points).to(device) * 2 * radius_rev - radius_rev), min= x_l,max = x_u)
        prop_x_centered_pts_set = prop_x_centered_pts_set.reshape(-1,1).to(device)
        
        prop_x_pts_set_e = total_free_e_derivative(net, prop_x_centered_pts_set, t3, min_raw_loss)
        prop_x_pts_set_e = net.scaler(prop_x_pts_set_e)
        
        prop_x_pts_set_e = prop_x_pts_set_e.reshape(x_prev.shape[0],n_points)
        '''
        
        #get the original g(last |given original prop) as the final term of acceptance ratio a
        compliment_prob_e_given_o_prop = one-prob_e_prop_o
        compliment_prob_e_given_o_prop_IC = compliment_prob_e_given_o_prop[:initial_size]
        compliment_prob_e_given_o_prop_dom = compliment_prob_e_given_o_prop[initial_size:]
        
        prop_std_rev_o_IC = const_std_IC *compliment_prob_e_given_o_prop_IC**.25
        prop_std_rev_o_dom = const_std_dom *compliment_prob_e_given_o_prop_dom**.25
        prop_std_rev_o = torch.cat([prop_std_rev_o_IC, prop_std_rev_o_dom], axis = 0)
        
        '''
        # Compute the maximum in each set
        
        #e_max_per_set_rev = torch.quantile(prop_x_pts_set_e, q = q, dim=1, keepdim = True).reshape(-1,1) #0.05 #.values ## simulation data x'
        '''
        compliment_prob_e_rev = one-e_max_per_set_rev**.1
        compliment_prob_e_rev_IC = compliment_prob_e_rev[:initial_size]
        compliment_prob_e_rev_dom = compliment_prob_e_rev[initial_size:]
        
        prop_std_rev_IC = const_std_IC *compliment_prob_e_rev_IC # **.5 
        prop_std_rev_dom = const_std_dom *compliment_prob_e_rev_dom # **.5
        prop_std_rev = torch.cat([prop_std_rev_IC, prop_std_rev_dom], axis = 0)
        '''
        '''
        #------------------------------- epsilon mass assign ----------------------------------------------------
        mass_eps_prop = torch.where( 
                              torch.abs(observed - e_max_per_set_rev) < epsilon,
                               1, 0
                              ) #epsilon 0.001 - 0.0001 #0.0001
        mass_eps_prev = torch.where(
                              torch.abs(observed - e_max_per_set)< epsilon,
                               1, 0
                              ) # changed 0.04
        #---------------------------------------------------------------------------------------------------------
        '''
        #------------------------------- epsilon mass assign ----------------------------------------------------
        mass_eps_prop = torch.where( 
                              kde_pairwise_similarity(observed, prop_x_pts_set_e, 1) < epsilon,
                               1, 0
                              ) #epsilon 0.001 - 0.0001 #0.0001
        mass_eps_prev = torch.where(
                              kde_pairwise_similarity(observed, x_prev_pts_set_e, 1)< epsilon,
                               1, 0
                              ) # changed 0.04
        #---------------------------------------------------------------------------------------------------------
        
        p_theta = prob_e_prop #total_free_e_derivative(net, prop_x, t2, min_raw_loss) #normal_pdf(x_prev, prop_x_o, prop_std_rev_o)
        p_thetap = prob_e #total_free_e_derivative(net, x_prev, t2, min_raw_loss) #normal_pdf(prop_x_o, x_prev, prop_std_o)
        
        #p_x = ( normal_pdf(x = x_prev,mean = prop_x , std= prop_std_rev ).to(device)/
        #       normal_pdf(x = prop_x,mean = x_prev , std=prop_std).to(device) )
        
        
        upper = (mass_eps_prop/mass_eps_prev * p_theta/p_thetap ) #*p_x
        
        with torch.autograd.no_grad():
            if i % 100 ==0:
                torch.save(torch.cat([prop_x, x_prev],axis=1), 'proposal_vs_prev.pt')
                #torch.save(torch.cat([prop_std, x_prev],axis=1), 'proposed_std.pt')
                #torch.save(torch.cat([prop_std_rev, x_prev],axis=1), 'proposed_std_rev.pt')
                #torch.save(torch.cat([prob_e_prop_o, prop_x_o],axis=1), 'prob_e_rev.pt')
                #torch.save(torch.cat([prob_e, x_prev],axis=1), 'prob_e.pt')
                
                #torch.save(torch.cat([p_given_last, x_prev],axis=1), 'pb_x_xprime.pt')
                #torch.save(torch.cat([p_given_o_proposal, prop_x_o],axis=1), 'pb_xprime_x.pt')
                torch.save(torch.cat([upper, x_prev],axis=1), 'upper.pt')
                #np.savetxt(f"{ctime}_epsilon.txt", epsilon)
                
            
        
        #upper = Variable(torch.from_numpy(upper).float(), requires_grad=True).to(device)
        mesure = PDE_lh_sampler4.random(n=size)[:,0].reshape(size,1) #torch.rand_like(prob_e)#np.random.uniform(0,1,(size,1) )
        mesure = Variable(torch.from_numpy(mesure).float(), requires_grad=True).to(device) #.cpu().numpy()
        #x_prev[mesure < upper] = prop_x[mesure < upper]
        with torch.no_grad():
            indices = ((mesure < one)&(mesure < upper)&(prop_x >= x_l) & (prop_x <= x_u)).squeeze()
            x_prev[indices] = prop_x[indices]
            
            accepted_IC += indices[:initial_size].sum().item()
            accepted_dom += indices[initial_size:].sum().item()
            #cumul_accepted += indices.sum().item()
            
            
            if i % 100 ==0:
                np.savetxt(f"{ctime}_std_log.txt", std_log, fmt="%.5f")
            # Adapt proposal std every 100 steps
            if i % update_std_iter ==0 and i > 30:
                acceptance_rate_IC = accepted_IC / (x_IC.shape[0] * update_std_iter)
                acceptance_rate_dom = accepted_dom / (x_dom.shape[0] * update_std_iter)
                '''
                mean_std_IC = torch.mean(prop_std_IC)
                mean_std_dom = torch.mean(prop_std_dom)
                
                #if acceptance_rate_IC < 0.30 :
                    #j *= .95
                if acceptance_rate_IC > 0.60  :
                    epsilon *= 10**(1/19)
                #if acceptance_rate_dom < 0.30 :
                    #j *= .95
                if acceptance_rate_dom > 0.60:
                    epsilon *= 10**(1/19)
                
                if i % 50 ==0:
                    std_log.append([acceptance_rate_IC, mean_std_IC.item(), acceptance_rate_dom, mean_std_dom.item()])
                '''
                accepted_IC = 0  # reset
                accepted_dom = 0  # reset
            
        i +=1
    with torch.autograd.no_grad():
        if epoch ==1:
            torch.save(torch.cat([x_prev[initial_size:], t],axis=1), f'{final_time}_adap_begin.pt')
        if epoch == 99 and not initialize_sampling:
            torch.save(torch.cat([x_prev[initial_size:], t],axis=1), f'{final_time}_adap_end.pt')
    return x_prev[:initial_size], x_prev[initial_size:]

def training_loop(net, time_slices, iterations, learning_rate, record_loss, print_loss):
    global epsilon
    global min_raw_loss
    
    # Domain boundary values
    x_l = net.x1_l
    x_u = net.x1_u
    
    #time starts at 0, ends at upper bouund updated in time_slices
    t_l = 0

    #numbers of sampling collocation points on each part
    lamda = 2/3
    IC_collocation = int(200*lamda)#150
    BC_collocation = int(200) #200
    pde_collocation = int(12000*lamda)#10000
    
    IC_adap_collocation = int(200*(1-lamda)) #50
    pde_adap_collocation = int(12000*(1-lamda))
    
    #sampler setting for LatinHypercube sampling
    IC_lh_sampler = qmc.LatinHypercube(d=1)
    IC_lh_sampler2 = qmc.LatinHypercube(d=1)
    #IC_lh_sampler3 = qmc.LatinHypercube(d=1)
    BC_lh_sampler = qmc.LatinHypercube(d=1)
    PDE_lh_sampler = qmc.LatinHypercube(d=2)
    #PDE_lh_sampler2 = qmc.LatinHypercube(d=2)
    PDE_lh_sampler3 = qmc.LatinHypercube(d=2)
    #PDE_lh_sampler4 = qmc.LatinHypercube(d=2)
    
    #update the learning rate as defined
    for g in net.optimizer.param_groups:
        g['lr'] = learning_rate
    
    #Iterate over time slices
    for final_time in time_slices:
        with torch.autograd.no_grad():
            print("\n\nCurrent End Time:", final_time, "Current Learning Rate: ", get_lr(net.optimizer))  
        
        global min_loss
        min_loss = 20
        global epoch
        for epoch in range(iterations):
            
            ##Define input points with LatinHypercube sampling
            x_IC = x_l + (x_u-x_l) *IC_lh_sampler.random(n=IC_collocation )
            #x_IC_scaled = qmc.scale(x_IC_sample, [x_l], [x_u])
            #x_IC_full = np.vstack((x_IC_scaled, [[x_l],[x_u]]))
            #np.random.shuffle(x_IC_full)
            #x_IC = x_IC_sample.reshape(IC_collocation,1) #x_l + (x_u-x_l) *IC_lh_sampler.random(n=IC_collocation).reshape(IC_collocation,1)
            #t_IC = np.random.uniform(low=t_l, high=t_l, size=(IC_collocation,1))
            
            input_x_IC = Variable(torch.from_numpy(x_IC).float(), requires_grad=True).to(device) #.detach()
            #input_x_IC = torch.where(input_x_IC<1,input_x_IC,-input_x_IC).to(device)
            #input_t_IC = Variable(torch.from_numpy(t_IC).float(), requires_grad=True).to(device)
            
            t_BC = t_l + (final_time-t_l) *BC_lh_sampler.random(n=BC_collocation).reshape(BC_collocation,1)
    
            input_t_BC = Variable(torch.from_numpy(t_BC).float(), requires_grad=True).to(device)
    
            x_domain = x_l + (x_u-x_l) *PDE_lh_sampler.random(n=pde_collocation )[:,0].reshape(pde_collocation,1)
            #x_domain_scaled = qmc.scale(x_domain_sample, [x_l], [x_u])
            #x_domain_full = np.vstack((x_domain_scaled, [[x_l],[x_u]]))
            #np.random.shuffle(x_domain_full)
            #x_domain = x_domain_sample.reshape(pde_collocation,1) # x_l + (x_u-x_l) *PDE_lh_sampler.random(n=pde_collocation)[:,0].reshape(pde_collocation,1)
            
            
            
            #t_domain = t_l + (final_time-t_l) *PDE_lh_sampler.random(n=pde_collocation)[:,1].reshape(pde_collocation,1)
            t_domain_exp = exp_time_sample(pde_collocation, t_l, final_time)
            
            input_x_domain = Variable(torch.from_numpy(x_domain).float(), requires_grad=True).to(device)
            
            #input_t_domain = Variable(torch.from_numpy(t_domain).float(), requires_grad=True).to(device)
            input_t_domain_exp = Variable(torch.from_numpy(t_domain_exp).float(), requires_grad=True).to(device)
            '''
            x_domain_r = x_l + (x_u-x_l) *PDE_lh_sampler2.random(n=pde_collocation)[:,0].reshape(pde_collocation,1)
            #t_domain_r = t_l + (final_time-t_l) *PDE_lh_sampler2.random(n=pde_collocation)[:,1].reshape(pde_collocation,1)
            t_domain_r_exp = exp_time_sample(pde_collocation, t_l, final_time)
            
            input_x_domain_r = Variable(torch.from_numpy(x_domain_r).float(), requires_grad=True).to(device)
            #input_t_domain_r = Variable(torch.from_numpy(t_domain_r).float(), requires_grad=True).to(device)
            input_t_domain_r_exp = Variable(torch.from_numpy(t_domain_r_exp).float(), requires_grad=True).to(device)
            '''
            if epoch == 0: #_k: Initializing e_adaptive in 0poch and then resampling for e_adaptive on the trained result from epoch1
                #Initialize the e-adaptive sampling on spatial domain
                
                x_IC_ad = x_l + (x_u-x_l) *IC_lh_sampler2.random(n=IC_adap_collocation )
                input_x_IC_ad = Variable(torch.from_numpy(x_IC_ad).float(), requires_grad=True).to(device) #.detach()
                
                x_domain_ad = x_l + (x_u-x_l) *PDE_lh_sampler3.random(n=pde_adap_collocation )[:,0].reshape(pde_adap_collocation,1)
                input_x_domain_ad = Variable(torch.from_numpy(x_domain_ad).float(), requires_grad=True).to(device)
                
                t_ad_exp = exp_time_sample(pde_adap_collocation, t_l, final_time)
                input_t_ad_exp = Variable(torch.from_numpy(t_ad_exp).float(), requires_grad=True).to(device)
                
                if final_time == 0.1:
                    x_IC_adap2, x_domain_adap2 = e_adaptive_sampling(net,IC_adap_collocation, pde_adap_collocation, input_x_IC_ad, input_x_domain_ad, None, True,final_time)
                    
                else:
                    x_IC_adap2, x_domain_adap2 = e_adaptive_sampling(net,IC_adap_collocation, pde_adap_collocation, input_x_IC_ad, input_x_domain_ad, input_t_ad_exp, True,final_time)
                    
                
            else:
                t_ad_exp = exp_time_sample(pde_adap_collocation, t_l, final_time)
                input_t_ad_exp = Variable(torch.from_numpy(t_ad_exp).float(), requires_grad=True).to(device)
                
                #Initialize the e-adaptive sampling on spatial domain
                x_IC_adap2, x_domain_adap2 = e_adaptive_sampling(net,IC_adap_collocation, pde_adap_collocation, input_x_IC_ad, input_x_domain_ad, input_t_ad_exp, False,final_time)
                
                
            input_t_ad_exp = Variable(torch.from_numpy(input_t_ad_exp.detach().cpu().numpy()).float(), requires_grad=True).to(device)
            #tensor grad stop by detach() and wrap again by Variable()
            #x_IC_adap2 = torch.where(x_IC_adap2<1, x_IC_adap2, -x_IC_adap2).detach()
            x_IC_adap2 = Variable(torch.from_numpy(x_IC_adap2.detach().cpu().numpy()).float(),requires_grad=True).to(device)
                
            
            #x_domain_adap2 = torch.where(x_domain_adap2<1,x_domain_adap2,-x_domain_adap2)
            x_domain_adap2 = Variable(torch.from_numpy(x_domain_adap2.detach().cpu().numpy()).float(),requires_grad=True).to(device)
            
            for i in list(range(0, int(pde_collocation//24)-1)):
                # initialize gradients to zero
                net.optimizer.zero_grad()
                '''   
                #input_t_ad = Variable(torch.from_numpy(input_t_ad).float(), requires_grad=True).to(device)
                x1 = input_x_domain_r[i*32:(i+1)*32]
                #if i % 2 != 0:
                    #x1 = -x1
                t1 = input_t_domain_r_exp[i*32:(i+1)*32]
                AC_mse_loss_adap = lossNSpde_rank(net, x1, t1, batch_size = 32)     
                '''
                    
                #Loss computation
                m = int(IC_collocation*24/pde_collocation)
                x2 = input_x_IC #[i*m:(i+1)*m]
                    
                k = int(IC_adap_collocation*24/pde_collocation)
                x2_adap = x_IC_adap2 #[i*k:(i+1)*k]
                #if i % 2 != 0:
                    #x2 = -x2
                    #x2_adap = -x2_adap
                u_IC_loss = lossIC(net, x2)
                u_IC_loss_ad = lossIC(net, x2_adap) #, u_IC_loss_mesh
                mse_IC = lamda*u_IC_loss + (1-lamda)*u_IC_loss_ad
                
                #Loss based on Boundary Condition (Containing No-Slip and Free-slip)
                mse_BC_u, mse_BC_u_x = lossBdry(net, input_t_BC) 
                mse_BC = mse_BC_u+ mse_BC_u_x 
                
                #Loss based on PDE
                x3 =input_x_domain[i*24:(i+1)*24]
                t3 =input_t_domain_exp[i*24:(i+1)*24]
                #if i % 2 != 0:
                    #x3 = -x3
                AC_mse= lossNSpde(net, x3, t3) 
    
                    
                n = int(pde_adap_collocation*24/pde_collocation)
                x4 = x_domain_adap2[i*n:(i+1)*n]
                t4 = input_t_ad_exp[i*n:(i+1)*n]
                #if i % 2 != 0:
                    #x4 = -x4
                AC_mse_ad = lossNSpde(net,x4 , t4)
                
                mse_PDE = lamda*AC_mse + (1-lamda)*AC_mse_ad*500 #+ AC_mse_loss_adap
                
                loss = (mse_BC + 1000*mse_IC + mse_PDE )
                raw_loss = (mse_BC + mse_IC + mse_PDE )

                
                loss.backward() #retain_graph=True
                

                
                def closure():
                    return loss

                # Gradient Norm Clipping
                total_norm = torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm = 1000, error_if_nonfinite=False) #note max norm is more art than science
                
                #Make Iterative Step
                net.optimizer.step() #net.optimizer.step(closure)
                
                




            
            with torch.autograd.no_grad():
                if epoch == 0 or epoch% 2 == 0: #print_loss
                    print("\nIteration:", epoch, "\tTotal Loss:", loss.data)
                    #,"\tadaptive IC Loss2: ", mse_IC_a2.item() , "\tadaptive CH PDE Loss2: ", CH_a2.item()
                    print("\tIC Loss: ", mse_IC.item(), 
                          "\t BC u Loss: ", mse_BC_u.item(), "\t BC u_x Loss: ", mse_BC_u_x.item(),"\tE-Adaptive IC Loss: ", u_IC_loss_ad.item(),
                          "\nAC PDE Loss: ", AC_mse.item(),"\tE-Adaptive PDE Loss: ", AC_mse_ad.item()
                          ) #, "\tloss adaptive PDE Loss: ", AC_mse_loss_adap.item()
                    print(f"\nGradient norm before clipping: {total_norm:.4e}")
                    if min_raw_loss > raw_loss:
                        min_raw_loss = raw_loss
                    if loss.cpu().detach().numpy() < min_loss:
                        min_loss = loss.cpu().detach().numpy()
                        torch.save(net.state_dict(), f"{ctime}_ES_min_loss_lr{get_lr(net.optimizer)}_t{final_time}.pt")
                        print('  *Saved ; Early Stopping for the Minimal Total Loss\n')
                        
                if epoch%1 == 0:
                    epsilon = np.append(epsilon, raw_loss.cpu().detach().numpy())
                    np.savetxt(f"{ctime}_epsilon.txt", epsilon)
                    torch.save(torch.cat([x_domain_adap2, input_t_ad_exp],axis=1), 'domain_edadp.pt')
                
                if epoch ==iterations -1:
                    #torch.save( torch.cat([input_x_domain, input_t_domain_exp],axis=1), 'domain_exp.pt')
                    #torch.save( torch.cat([input_x_domain, input_t_domain_exp],axis=1), 'domain.pt')
                    
                    torch.save(net.state_dict(), f"{ctime}_lr{get_lr(net.optimizer)}_t{final_time}.pt")
                
                

create_network(load, loadfile) #*500 on adaptive term / comment out scaler /std 0.07(later noise incrase) / radius pts 20(increase this later) / sampling iteration 300 / increased lr


Date of Today :  8  / 7 
Hour :  18
Training PDE


Training Pass 1


Current End Time: 0.1 Current Learning Rate:  0.001

Iteration: 0 	Total Loss: tensor(192.6782, device='cuda:0')
	IC Loss:  0.15677618980407715 	 BC u Loss:  1.2029638290405273 	 BC u_x Loss:  0.06873273104429245 	E-Adaptive IC Loss:  0.13427947461605072 
AC PDE Loss:  0.1591668576002121 	E-Adaptive PDE Loss:  0.20714536309242249

Gradient norm before clipping: 1.1802e+04

Iteration: 2 	Total Loss: tensor(9.0936, device='cuda:0')
	IC Loss:  0.006883900612592697 	 BC u Loss:  0.054375190287828445 	 BC u_x Loss:  0.005233517847955227 	E-Adaptive IC Loss:  0.007439926266670227 
AC PDE Loss:  0.014981667511165142 	E-Adaptive PDE Loss:  0.012840867042541504

Gradient norm before clipping: 4.1800e+03
  *Saved ; Early Stopping for the Minimal Total Loss


Iteration: 4 	Total Loss: tensor(17.5098, device='cuda:0')
	IC Loss:  0.011829838156700134 	 BC u Loss:  0.0005325556267052889 	 BC u_x Loss:  0.006031303200870752 	E-Adapt